In [ ]:
!pip install flask --break-system-packages
!pip install flask-restx  --break-system-packages

In [ ]:
import nest_asyncio
from flask import Flask, jsonify, request
from werkzeug.serving import run_simple

nest_asyncio.apply()

app = Flask(__name__) # Application Tier app

#http://localhost:5000/hello
@app.route("/hello", methods=["GET"])
def hello_world():
    message_data = {"message": "Hello, World!"} 

    return jsonify(message_data), 200 # data = {"message": "Hello, World!"} => jsonify(data) =  {\"message\": \"Hello, World!\"}

# http://localhost:5000/greet/adam => greet(name="adam") # we call this endpoint with path variable
@app.route("/greet/<string:name>", methods=["GET"])
def greet(name: str):
    message_data = {"message": f"Hello, {name}!"} 
    return jsonify(message_data), 200# data = {"message": "Hello, World!"} => jsonify(data) =  {\"message\": \"Hello, Adam!\"}

# http://localhost:5000/query
# http://localhost:5000/query?name=adam
# http://localhost:5000/query?name=adam&age=37
@app.route("/query", methods=["GET"])
def query():
    name = request.args.get("name", "Guest") # if name is none then name =  Guest
    age = request.args.get("age")
    
    data = {"message": f"Hello, {name}!"}
    
    if age: # if age is not none
        data = {"message": f"Hello, {name}! You are {age} years old."}
        
    return jsonify(data), 200
        
        
    

# localhost = 127.0.0.1, 0.0.0.0
run_simple('localhost', 5000, app, use_debugger=True)

### Posts Management System 

#### RESTful API Features:
- POST /post: Create a new post.
- GET /post/int:post_id: Retrieve a specific post by ID.
- PUT /post/int:post_id: Update an existing post by ID.
- DELETE /post/int:post_id: Delete a specific post by ID.
 

####  Models  [data (list of dictionary | database table in the form of  ORM)] 
- id (int): Unique identifier for the post.
- title (str): Title of the post.
- content (str): Content of the post.

```python
# in memory
 post = {"id": 1, "title": "some", "content": "1111"}
 posts = {
    "1" : {"id": 1, "title": "somasdfase", "content": "1asdfa111"},
    "2" : {"id": 2, "title": "somdasfde", "content": "11afdasdfa11"}
 }
 # in database
 class Post(db.Model):
    __tablename__= "posts"
    id = ....
    title = ...
    content = ....
```
### Posts Table

| ID  | Title       | Content         |
|-----|-------------|-----------------|
| 1   | somasdfase  | 1asdfa111       |
| 2   | somdasfde   | 11afdasdfa11    |


In [ ]:
from http import HTTPStatus
import nest_asyncio
from werkzeug.serving import run_simple
from flask import Flask, jsonify, request, abort

nest_asyncio.apply()

app = Flask(__name__)

posts  = {}    

# address http://localhost:4000
# endpoint  /posts 
# route http://localhost:4000/posts  
# method POST
@app.route("/posts", methods=["POST"])
def create_post():
    data = request.get_json() # Get JSON payload from  request
    
    if not data or 'id' not in data or 'title' not in data or 'content' not in data:
        response_message = {"error": "Invalid input"}
        response_status_code = HTTPStatus.BAD_REQUEST
        
        return jsonify(response_message), response_status_code
    
    post_id = data['id']
    post_title = data['title']
    post_content = data['content']
    
    if post_id in posts:
        response_message = {"error": "Post with this ID already exists"}
        response_status_code = HTTPStatus.CONFLICT
        
        return jsonify(response_message), response_status_code
    
    posts[post_id] = {"id":post_id, "title": post_title, "content": post_content} 
    response_message = {"success": "Post created", "data": data}
    response_status_code = HTTPStatus.CREATED
        
    return jsonify(response_message), response_status_code
    


# address http://localhost:4000
# endpoint  /posts/{post_id}
# route http://localhost:4000/posts/{post_id}
# method GET
@app.route("/posts/<int:post_id>", methods=["GET"])
def get_posts(post_id: int):
    if post_id not in posts: 
        response_message = {"error": "Post not found"}
        response_status_code = HTTPStatus.NOT_FOUND
        
        return jsonify(response_message), response_status_code
    
    response_message = {"id": post_id, "post": posts[post_id]}
    response_status_code = HTTPStatus.OK
    
    return jsonify(response_message), response_status_code

# address http://localhost:4000
# endpoint  /posts/{post_id}
# route http://localhost:4000/posts/{post_id}
# method PUT 
@app.route("/posts/<int:post_id>", methods=["PUT"])
def update_post(post_id: int):
    if post_id not in posts: 
        response_message = {"error": "Post not found"}
        response_status_code = HTTPStatus.NOT_FOUND
        
        return jsonify(response_message), response_status_code
    
    data = request.get_json() # Get JSON payload from request
    
    if not data or 'title' not in data or 'content' not in data:
        response_message = {"error": "Invalid input"}
        response_status_code = HTTPStatus.BAD_REQUEST
        
        return jsonify(response_message), response_status_code
    
    posts[post_id] =  {"title": data['title'], "content": data['content']}
    
    response_message = {"id": post_id, "post": posts[post_id]}
    response_status_code = HTTPStatus.OK
    
    return jsonify(response_message), response_status_code
    

# address http://localhost:4000
# endpoint  /posts/{post_id}
# route http://localhost:4000/posts/{post_id}
# method DELETE
@app.route("/posts/<int:post_id>", methods=["DELETE"])
def delete_post(post_id: int):
    if post_id not in posts: 
        response_message = {"error": "Post not found"}
        response_status_code = HTTPStatus.NOT_FOUND
        
        return jsonify(response_message), response_status_code
    
    del posts[post_id]

    response_message = {"success": "Post deleted"}
    response_status_code = HTTPStatus.NO_CONTENT
    
    return jsonify(response_message), response_status_code
    
 


# Run Flask in Jupyter Notebook
run_simple('localhost', 4000, app)

Comment Management System 

#### RESTful API Features:
- POST /comment: Create a new comment.
- GET /comment/int:comment_id: Retrieve a specific comment by ID.
- PUT /comment/int:comment_id: Update an existing comment by ID.
- DELETE /comment/int:comment_id: Delete a specific comment by ID. 
 
####  Model  
- id (int): Unique identifier for the comment.
- content (str): The text content of the comment.
- post_id (int): The ID of the post this comment is associated with.



```python
# in memory
 comment = {"id": 1, "content": "some", "post_id": 1}
 comments = {
    "1" : {"id": 1, "content": "somasdfase", "post_id": 1},
    "2" : {"id": 2, "content": "somdasfde", "post_id": 2}
 }
 # in database
 class Comment(db.Model):
    __tablename__= "posts"
    id = ....
    content = ...
    id = ....
```
### Posts Table

| ID  | content     | post_id         |
|-----|-------------|-----------------|
| 1   | somasdfase  | 1               |
| 2   | somdasfde   | 2               |


In [ ]:
from http import HTTPStatus
import nest_asyncio
from werkzeug.serving import run_simple
from flask import Flask, jsonify, request, abort

nest_asyncio.apply()

app = Flask(__name__)

# In-memory storage for comments
comments = {}    


# address http://localhost:5000
# endpoint  /comment
# route http://localhost:5000/comment/
# method POST
@app.route("/comment", methods=["POST"])
def create_comment():
    data = request.get_json()
    if not data or 'id' not in data or 'content' not in data or 'post_id' not in data:
        response_message = {"error": "Invalid input"}
        response_status_code = HTTPStatus.BAD_REQUEST
        
        return jsonify(response_message), response_status_code
    
    comment_id = data['id'] 
    
    if comment_id in comments:
        response_message = {"error": "Comment with this ID already exists"}
        response_status_code = HTTPStatus.CONFLICT
        
        return jsonify(response_message), response_status_code
        
    comments[comment_id] = data 
    
    response_message = {"success": "Comment created", "data": data}
    response_status_code = HTTPStatus.CREATED
        
    return jsonify(response_message), response_status_code
    
    
# address http://localhost:5000
# endpoint  /comment/{comment_id}
# route http:/localhost:5000/comment/{comment_id}
# method GET
@app.route("/comment/<int:comment_id>", methods=["GET"])
def get_comment(comment_id: int):
    if comment_id not in comments: 
        response_message = {"error": "Comment not found"}
        response_status_code = HTTPStatus.NOT_FOUND
        
        return jsonify(response_message), response_status_code
    
    response_message = {"id": comment_id, "comment": comments[comment_id]}
    response_status_code = HTTPStatus.OK
    
    return jsonify(response_message), response_status_code
# address http://localhost:5000
# endpoint  /comment/{comment_id}
# route http:/localhost:5000/comment/{comment_id}
# method PUT
@app.route("/comment/<int:comment_id>", methods=["PUT"])
def update_comment(comment_id:int):
    if comment_id not in comments: 
        response_message = {"error": "Post not found"}
        response_status_code = HTTPStatus.NOT_FOUND
        
        return jsonify(response_message), response_status_code
    
    data = request.get_json() # Get JSON payload from request
    
    if not data or 'content' not in data or 'post_id' not in data:
        response_message = {"error": "Invalid input"}
        response_status_code = HTTPStatus.BAD_REQUEST
        
        return jsonify(response_message), response_status_code
    
    comments[comment_id] =  {"content": data['content'], "post_id": data['post_id']}
    
    response_message = {"id": comment_id, "comment": comments[comment_id]}
    response_status_code = HTTPStatus.OK
    
    return jsonify(response_message), response_status_code

# address http://localhost:5000
# endpoint  /comment/{comment_id}
# route http:/localhost:5000/comment/{comment_id}
# method DELETE
@app.route("/comment/<int:comment_id>", methods=["DELETE"])
def delete_comment(comment_id:int): 
    if comment_id not in comments: 
        response_message = {"error": "Comment not found"}
        response_status_code = HTTPStatus.NOT_FOUND
        
        return jsonify(response_message), response_status_code
    
    del comments[comment_id]

    response_message = {"success": "Comment deleted"}
    response_status_code = HTTPStatus.NO_CONTENT
    
    return jsonify(response_message), response_status_code
    

# Run Flask in Jupyter Notebook
run_simple('localhost', 5000, app)


###  **Comment Posting System**
 
#### RESTful API Features:
##### Post Routes
- POST /post: Create a new post.
- GET /post/int:post_id: Retrieve a specific post by ID.
- PUT /post/int:post_id: Update a specific post.
- DELETE /post/int:post_id: Delete a specific post.
- GET /posts: Retrieve all posts.

##### Comment  Routes
- POST /comment: Create a new comment.
- GET /comment/int:comment_id: Retrieve a specific comment by ID.
- PUT /comment/int:comment_id: Update a specific comment.
- DELETE /comment/int:comment_id: Delete a specific comment.
- GET /comments/int:post_id: Retrieve all comments for a specific post.

#### Models:
- **Post**: Represents a blog post.
  - `id`: Unique identifier for the post.
  - `title`: Title of the post.
  - `content`: Content of the post.

- **Comment**: Represents a comment on a post.
  - `id`: Unique identifier for the comment.
  - `content`: Content of the comment.
  - `post_id`: Identifier of the associated post.


In [20]:
import nest_asyncio
from werkzeug.serving import run_simple
from flask import Flask, Blueprint, jsonify, request, abort


nest_asyncio.apply()

# Initialize Flask app
app = Flask(__name__)

# In-memory storage
posts = {}
comments = {}

# Blueprints
posts_bp =  Blueprint('posts', __name__, url_prefix='/post')
comment_pb = Blueprint('comments', __name__, url_prefix='/comment')

# posts_bp 
@posts_bp.route('', methods=["POST"]) 
def create_post():
    data = request.get_json() # Get JSON payload from  request
    
    if not data or 'id' not in data or 'title' not in data or 'content' not in data:
        response_message = {"error": "Invalid input"}
        response_status_code = HTTPStatus.BAD_REQUEST
        
        return jsonify(response_message), response_status_code
    
    post_id = data['id']
    post_title = data['title']
    post_content = data['content']
    
    if post_id in posts:
        response_message = {"error": "Post with this ID already exists"}
        response_status_code = HTTPStatus.CONFLICT
        
        return jsonify(response_message), response_status_code
    
    posts[post_id] = {"id":post_id, "title": post_title, "content": post_content} 
    response_message = {"success": "Post created", "data": data}
    response_status_code = HTTPStatus.CREATED
        
    return jsonify(response_message), response_status_code
    


@posts_bp.route('<int:post_id>', methods=["GET"])
def get_post(post_id: int):
    if post_id not in posts: 
        response_message = {"error": "Post not found"}
        response_status_code = HTTPStatus.NOT_FOUND
        
        return jsonify(response_message), response_status_code
    
    response_message = {"id": post_id, "post": posts[post_id]}
    response_status_code = HTTPStatus.OK
    
    return jsonify(response_message), response_status_code


@posts_bp.route('<int:post_id>', methods=["PUT"])
def update_post(post_id: int):
    if post_id not in posts: 
        response_message = {"error": "Post not found"}
        response_status_code = HTTPStatus.NOT_FOUND
        
        return jsonify(response_message), response_status_code
    
    data = request.get_json() # Get JSON payload from request
    
    if not data or 'title' not in data or 'content' not in data:
        response_message = {"error": "Invalid input"}
        response_status_code = HTTPStatus.BAD_REQUEST
        
        return jsonify(response_message), response_status_code
    
    posts[post_id] =  {"title": data['title'], "content": data['content']}
    
    response_message = {"id": post_id, "post": posts[post_id]}
    response_status_code = HTTPStatus.OK
    
    return jsonify(response_message), response_status_code


@posts_bp.route('<int:post_id>', methods=["DELETE"])
def delete_post(post_id: int):
    if post_id not in posts: 
        response_message = {"error": "Post not found"}
        response_status_code = HTTPStatus.NOT_FOUND
        
        return jsonify(response_message), response_status_code
    
    del posts[post_id]

    response_message = {"success": "Post deleted"}
    response_status_code = HTTPStatus.NO_CONTENT
    
    return jsonify(response_message), response_status_code
    
 
# comment_pb
@comment_pb.route('', methods=["POST"])
def create_comment():
    data = request.get_json()
    if not data or 'id' not in data or 'content' not in data or 'post_id' not in data:
        response_message = {"error": "Invalid input"}
        response_status_code = HTTPStatus.BAD_REQUEST
        
        return jsonify(response_message), response_status_code
    
    comment_id = data['id'] 
    
    if comment_id in comments:
        response_message = {"error": "Comment with this ID already exists"}
        response_status_code = HTTPStatus.CONFLICT
        
        return jsonify(response_message), response_status_code
        
    comments[comment_id] = data 
    
    response_message = {"success": "Comment created", "data": data}
    response_status_code = HTTPStatus.CREATED
        
    return jsonify(response_message), response_status_code


@comment_pb.route('<int:comment_id>', methods=["GET"])
def get_comment(comment_id: int):
    if comment_id not in comments: 
        response_message = {"error": "Comment not found"}
        response_status_code = HTTPStatus.NOT_FOUND
        
        return jsonify(response_message), response_status_code
    
    response_message = {"id": comment_id, "comment": comments[comment_id]}
    response_status_code = HTTPStatus.OK
    
    return jsonify(response_message), response_status_code


@comment_pb.route('<int:comment_id>', methods=["PUT"])
def update_comment(comment_id: int):
    if comment_id not in comments: 
        response_message = {"error": "Post not found"}
        response_status_code = HTTPStatus.NOT_FOUND
        
        return jsonify(response_message), response_status_code
    
    data = request.get_json() # Get JSON payload from request
    
    if not data or 'content' not in data or 'post_id' not in data:
        response_message = {"error": "Invalid input"}
        response_status_code = HTTPStatus.BAD_REQUEST
        
        return jsonify(response_message), response_status_code
    
    comments[comment_id] =  {"content": data['content'], "post_id": data['post_id']}
    
    response_message = {"id": comment_id, "comment": comments[comment_id]}
    response_status_code = HTTPStatus.OK
    
    return jsonify(response_message), response_status_code


@comment_pb.route('<int:comment_id>', methods=["DELETE"])
def delete_comment(comment_id: int):
    if comment_id not in comments: 
        response_message = {"error": "Comment not found"}
        response_status_code = HTTPStatus.NOT_FOUND
        
        return jsonify(response_message), response_status_code
    
    del comments[comment_id]

    response_message = {"success": "Comment deleted"}
    response_status_code = HTTPStatus.NO_CONTENT
    
    return jsonify(response_message), response_status_code
    



# Register Blueprints to app
app.register_blueprint(posts_bp)
app.register_blueprint(comment_pb)

# Run Flask in Jupyter Notebook
run_simple('localhost', 5000, app)

 * Running on http://localhost:5000
Press CTRL+C to quit
